<p align = "center">
    <img src="img/cinta.png" />
</p>

<p align = "center">
    <img src="img/logo.png" />
</p>


# **Maestría en Ciencia de Datos**
## Temas Selectos de Modelado. Primavera 2022.

### **Equipo 1** 

- Nyrma Paulina Hernández Trejo
- Aide Jazmín González Cruz
- Joel Jaramillo Pacheco
- Jesús Enrique Miranda Blanco

<br>

<div align="center"><h1 style="font-weight:bold; font-size: 40px">Práctica 2</h1></div>


# **Segunda parte**

Se muestra la ejecución del paquete en el que se muestra el mejoramiento realizado. Para ello se cargan las librerías necesarias:

In [1]:
from IPython.display import display, HTML, Image
from datetime import date
import pandas as pd
import random
import numpy as np 
import networkx as nx
import time

Y se muestran las características de la máquina donde se corrió el código:

In [2]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          2
On-line CPU(s) list:             0,1
Thread(s) per core:              2
Core(s) per socket:              1
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
Stepping:                        7
CPU MHz:                         3100.063
BogoMIPS:                        4999.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       32 KiB
L1i cache:                       32 KiB
L2 cache:                        1 MiB
L3 cache:                        35.8 MiB
NUMA node0 CPU(s):               0,

In [3]:
%%bash
 sudo apt-get update

Hit:1 https://deb.nodesource.com/node_16.x focal InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...


In [4]:
%%bash
sudo apt-get -y install lshw

Reading package lists...
Building dependency tree...
Reading state information...
lshw is already the newest version (02.18.85-0.3ubuntu2.20.04.1).
0 upgraded, 0 newly installed, 0 to remove and 120 not upgraded.


In [5]:
%%bash
sudo lshw -C memory

  *-memory
       description: System memory
       physical id: 0
       size: 15GiB


In [6]:
%%bash
uname -ar #r for kernel, a for all

Linux 53c16d50e897 5.4.0-1072-aws #77~18.04.1-Ubuntu SMP Thu Apr 7 21:38:47 UTC 2022 x86_64 x86_64 x86_64 GNU/Linux


- **Código original**

In [7]:
def get_data(data_dir, date=date.today().strftime("%Y-%m-%d"), tam_data=100, price='Open'):
    """
    Obtiene datos de un directorio de una fecha dada
    param:
    data_dir: directorio de datos
    date: fecha a analizar, por default toma la fecha actual si no se especifica
    tam_data: analiza cierto tamaño del dataset de forma aleatoria, por default son 100 monedas a analizar
    price: selecciona con que precio se hará el ejercicio(High,Low,Open,Close), por default es Open
    return:
    dataframe: con datos especificados
    """
    df = pd.read_csv(data_dir)
    df_date = df[df['Date']==date]
    df_date = df_date[df_date[price] > 0]
    
    if(tam_data == None):
        df_random = df_date
    else:
        df_random = df_date.sample(n = tam_data)
    
    
    df_random = df_random.reset_index()
    data = df_random[["ticker", price]]
    data.columns = ['Símbolo', 'Precio']
    
    return data    

def exchange_rate_matrix(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1[['Precio']].values[i]*(1+random.uniform(0,max_spread_pct))
    c1.drop(columns=['Precio'],inplace=True)
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1
    return c1

def log_transformed_rep(data):
    """
    Log-Transformed Representations
    param:
        dataframe
    return:
        dataframe
    """
    
    df_ln = np.round(-np.log(data),2)
    return df_ln

def create_grap(data):
    """
    Crea grafo a partir de los datos de cripomonedas y precio
    param:
        dataframe
    return:
        grafo
    """
    df = exchange_rate_matrix(data)
    df_ln = log_transformed_rep(df)
    
    n = df_ln.shape[0]
    
    edge = []
    # Covert to formatto use un graph
    for i in range(n):
        for j in range(n):
            if (i != j):
                edge.append([str(i), str(j), df_ln.loc[i][j]])

    G = nx.DiGraph()        
    G.add_weighted_edges_from(edge)
    
    return G

def bf_negative_cycle(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph
    edges = []
    for edge in graph.edges().data():
        if ~np.isnan(edge[2]['weight']):
            edges.append(edge)

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:  
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    return cycle.reverse()

- **Código perfilado**

In [8]:
def exchange_rate_matrix2(data):
    """
    Exchange Rate Matrix Representation
    param:
        dataframe
    return:
        dataframe
    """
    n = data.shape[0]

    max_spread_pct = 0.05 # maximum bid-ask spread in pct of bid, 0.05 for 5%

    c1 = data[['Precio']]
    aux = c1.copy()
    random.seed(10)
    for i in range(n):
        c1[i] = aux/c1.loc[i]['Precio']*(1+random.uniform(0,max_spread_pct))
        
    c1.drop(columns=['Precio'],inplace=True)
    
    for i in range(len(c1.index)):
        for j in range(len(c1.columns)):
            if i==j:
                c1.loc[i,j] = 1    
    return c1

def create_grap2(data):
    """
    Crea grafo a partir de los datos de cripomonedas y precio
    param:
        dataframe
    return:
        grafo
    """
    df = exchange_rate_matrix2(data)
    df_ln = log_transformed_rep(df)
    
    n = df_ln.shape[0]
    
    edge = []
    # Covert to formatto use un graph
    for i in range(n):
        for j in range(n):
            if (i != j):
                edge.append([str(i), str(j), df_ln.loc[i][j]])

    G = nx.DiGraph()        
    G.add_weighted_edges_from(edge)
    
    return G

def bf_negative_cycle2(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    n_nodes = len(graph.nodes)
    
    if node_ini is not None:
        assert node_ini <= n_nodes, f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {n_nodes}."
        n_nodes = node_ini
                    
    n = n_nodes + 1
    # Remove nan borders inside graph
    edges = [edge for edge in graph.edges().data() if ~np.isnan(edge[2]['weight'])]

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))
        
    # Initialize distances of nodes and predecessors
    # https://codingdeekshi.com/initialize-an-array-in-python/
    distance= [distance_ini ]*n
    distance[n_nodes] = 0  
    predecessors = [-1]*n 
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    return cycle.reverse()

## **Tiempo**

Se muestra la corrida del paquete anterior vs el perfilado con *time* y *timeit*

### ***Time***

- Código original proceso completo

In [9]:
start_time_c = time.time()
df = get_data('data/historical_data.csv', '2022-05-12')
G = create_grap(df)
start_time_m = time.time()
bf_negative_cycle(G)
end_time = time.time()
secs_c = end_time-start_time_c
secs_m = end_time-start_time_m
print("Arbitrage Identification Cycle in Crypto Trading tomó: ",secs_c,"segundos")
print("Bellman ford tomó: ",secs_m,"segundos")

Arbitrage Identification Cycle in Crypto Trading tomó:  3.929471015930176 segundos
Bellman ford tomó:  2.6580326557159424 segundos


- Código perfilado proceso completo

In [10]:
start_time_c = time.time()
df = get_data('data/historical_data.csv', '2022-05-12')
G = create_grap2(df)
start_time_m = time.time()
bf_negative_cycle2(G)
end_time = time.time()
secs_c = end_time-start_time_c
secs_m = end_time-start_time_m
print("Arbitrage Identification Cycle in Crypto Trading tomó: ",secs_c,"segundos")
print("Bellman ford tomó: ",secs_m,"segundos")

Arbitrage Identification Cycle in Crypto Trading tomó:  3.3247666358947754 segundos
Bellman ford tomó:  2.094679832458496 segundos


### ***Timeit***

- Bellman ford trabajado en prácticas anteriores

In [11]:
%timeit bf_negative_cycle(G)

2.67 s ± 13.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- Bellman ford perfilado

In [12]:
%timeit bf_negative_cycle2(G)

2.04 s ± 182 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### ***cProfile***

In [13]:
! pip install cProfile

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement cProfile (from versions: none)
ERROR: No matching distribution found for cProfile
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [14]:
import cProfile

- Bellman ford trabajado en prácticas anteriores

In [15]:
cprof = cProfile.Profile()
cprof.enable()
res = bf_negative_cycle(G)
cprof.disable()
cprof.print_stats(sort='cumtime')

         29979 function calls (29977 primitive calls) in 1.371 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.371    0.685 interactiveshell.py:3215(run_code)
        2    0.000    0.000    1.371    0.685 {built-in method builtins.exec}
        1    0.000    0.000    1.371    1.371 1203246769.py:3(<module>)
        1    1.362    1.362    1.371    1.371 2869294132.py:89(bf_negative_cycle)
     9901    0.005    0.000    0.008    0.000 reportviews.py:726(<genexpr>)
     9900    0.003    0.000    0.003    0.000 reportviews.py:712(<lambda>)
    10003    0.001    0.000    0.001    0.000 {method 'append' of 'list' objects}
        2    0.000    0.000    0.000    0.000 codeop.py:142(__call__)
        2    0.000    0.000    0.000    0.000 numeric.py:149(ones)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        2    0.000    0.000    0.000    0.000 <__array_funct

- Bellman ford perfilado

In [16]:
cprof = cProfile.Profile()
cprof.enable()
res = bf_negative_cycle2(G)
cprof.disable()
cprof.print_stats(sort='cumtime')

         20063 function calls (20062 primitive calls) in 1.089 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    1.088    0.544 interactiveshell.py:3215(run_code)
        2    0.000    0.000    1.088    0.544 {built-in method builtins.exec}
        1    0.000    0.000    1.088    1.088 1890767263.py:3(<module>)
        1    1.056    1.056    1.088    1.088 3611830932.py:52(bf_negative_cycle2)
        1    0.027    0.027    0.033    0.033 3611830932.py:64(<listcomp>)
     9901    0.004    0.000    0.006    0.000 reportviews.py:726(<genexpr>)
     9900    0.002    0.000    0.002    0.000 reportviews.py:712(<lambda>)
        2    0.000    0.000    0.000    0.000 codeop.py:142(__call__)
        2    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
      101    0.000    0.000    0.000    0.000 {method 'items' of 'dict' objects}
        2    0.000    0.000    0.000    0.000 contex

### ***line_profiler***

In [17]:
! pip install line_profiler

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
import line_profiler

- Bellman ford trabajado en prácticas anteriores

In [19]:
line_prof = line_profiler.LineProfiler()
print(line_prof(bf_negative_cycle)(G))
print(line_prof.print_stats())

None
Timer unit: 1e-06 s

Total time: 4.06207 s
File: /tmp/ipykernel_2275/2869294132.py
Function: bf_negative_cycle at line 89

Line #      Hits         Time  Per Hit   % Time  Line Contents
    89                                           def bf_negative_cycle(graph, node_ini=None, distance_ini=np.inf):
    90                                               
    91         1          3.0      3.0      0.0      assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    92                                               
    93         1          1.0      1.0      0.0      if node_ini is None:
    94         1         47.0     47.0      0.0          n_nodes = len(graph.nodes())
    95                                               else:
    96                                                   assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
    97      

- Bellman ford perfilado

In [20]:
line_prof = line_profiler.LineProfiler()
print(line_prof(bf_negative_cycle2)(G))
print(line_prof.print_stats())

None
Timer unit: 1e-06 s

Total time: 4.12395 s
File: /tmp/ipykernel_2275/3611830932.py
Function: bf_negative_cycle2 at line 52

Line #      Hits         Time  Per Hit   % Time  Line Contents
    52                                           def bf_negative_cycle2(graph, node_ini=None, distance_ini=np.inf):
    53                                               
    54         1          2.0      2.0      0.0      assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    55                                               
    56         1         13.0     13.0      0.0      n_nodes = len(graph.nodes)
    57                                               
    58         1          2.0      2.0      0.0      if node_ini is not None:
    59                                                   assert node_ini <= n_nodes, f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {n_nodes}."
    60                             

## **Memoria**

Se muestra la corridas con los paquetes para ver el mejoramiento usando *memory_profiler*

In [21]:
from memory_profiler import memory_usage

- Bellman ford trabajado en prácticas anteriores

In [22]:
t = (bf_negative_cycle, (G,1,100000))
print(memory_usage(t, max_usage=True))

120.125


In [23]:
start_mem = memory_usage(max_usage=True)
res = memory_usage(t, max_usage=True, retval=True)
print('start mem', start_mem)
print('max mem', res[0])
print('used mem', res[0]-start_mem)
print('fun output', res[1])

start mem 120.1640625
max mem 120.1640625
used mem 0.0
fun output None


- Bellman ford perfilado

In [24]:
t = (bf_negative_cycle2, (G,None,100000))
print(memory_usage(t, max_usage=True))

120.1640625


In [25]:
start_mem = memory_usage(max_usage=True)
res = memory_usage(t, max_usage=True, retval=True)
print('start mem', start_mem)
print('max mem', res[0])
print('used mem', res[0]-start_mem)
print('fun output', res[1])

start mem 120.171875
max mem 120.171875
used mem 0.0
fun output None


## CPU

In [26]:
%%bash
 sudo apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 https://deb.nodesource.com/node_16.x focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...


In [27]:
%%bash
 sudo apt install linux-tools-common

Reading package lists...
Building dependency tree...
Reading state information...
linux-tools-common is already the newest version (5.4.0-110.124).
0 upgraded, 0 newly installed, 0 to remove and 120 not upgraded.


In [28]:
%%bash
 sudo apt-get -y install linux-tools-5.4.0-1072-aws

Reading package lists...
Building dependency tree...
Reading state information...
linux-tools-5.4.0-1072-aws is already the newest version (5.4.0-1072.77).
0 upgraded, 0 newly installed, 0 to remove and 120 not upgraded.


- Bellman ford trabajado en prácticas anteriores

In [29]:
%%bash
 perf stat -S -e cycles,instructions,cache-references,cache-misses -r 20 python3 src/opt2/pipeline.py


 Performance counter stats for 'python3 src/opt2/pipeline.py' (20 runs):

   <not supported>      cycles                                                      
   <not supported>      instructions                                                
   <not supported>      cache-references                                            
   <not supported>      cache-misses                                                

           2.52719 +- 0.00924 seconds time elapsed  ( +-  0.37% )



- Bellman ford perfilado

In [30]:
%%bash
 perf stat -S -e cycles,instructions,cache-references,cache-misses -r 20 python3 src/opt2/pipeline2.py


 Performance counter stats for 'python3 src/opt2/pipeline2.py' (20 runs):

   <not supported>      cycles                                                      
   <not supported>      instructions                                                
   <not supported>      cache-references                                            
   <not supported>      cache-misses                                                

           2.22617 +- 0.00621 seconds time elapsed  ( +-  0.28% )



### Estadísticas por core

- Bellman ford trabajado en prácticas anteriores

In [31]:
%%bash
 perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 20 python3 src/opt2/pipeline.py


 Performance counter stats for 'system wide' (20 runs):

CPU0         <not supported>      cycles                                                      
CPU1         <not supported>      cycles                                                      
CPU0         <not supported>      instructions                                                
CPU1         <not supported>      instructions                                                
CPU0         <not supported>      cache-references                                            
CPU1         <not supported>      cache-references                                            
CPU0         <not supported>      cache-misses                                                
CPU1         <not supported>      cache-misses                                                

           2.52596 +- 0.00867 seconds time elapsed  ( +-  0.34% )



- Bellman ford perfilado

In [32]:
%%bash
 perf stat -S --all-cpus -A -e cycles,instructions,cache-references,cache-misses -r 20 python3 src/opt2/pipeline2.py


 Performance counter stats for 'system wide' (20 runs):

CPU0         <not supported>      cycles                                                      
CPU1         <not supported>      cycles                                                      
CPU0         <not supported>      instructions                                                
CPU1         <not supported>      instructions                                                
CPU0         <not supported>      cache-references                                            
CPU1         <not supported>      cache-references                                            
CPU0         <not supported>      cache-misses                                                
CPU1         <not supported>      cache-misses                                                

           2.21754 +- 0.00313 seconds time elapsed  ( +-  0.14% )



### Estadísticas

In [33]:
%%bash
 perf stat -S -r 20 python3 src/opt2/pipeline.py


 Performance counter stats for 'python3 src/opt2/pipeline.py' (20 runs):

           2575.60 msec task-clock                #    1.019 CPUs utilized            ( +-  0.33% )
                27      context-switches          #    0.010 K/sec                    ( +-  2.43% )
                 0      cpu-migrations            #    0.000 K/sec                    ( +- 58.49% )
             26093      page-faults               #    0.010 M/sec                    ( +-  0.04% )
   <not supported>      cycles                                                      
   <not supported>      instructions                                                
   <not supported>      branches                                                    
   <not supported>      branch-misses                                               

           2.52652 +- 0.00862 seconds time elapsed  ( +-  0.34% )



In [34]:
%%bash
 perf stat -S -r 20 python3 src/opt2/pipeline2.py


 Performance counter stats for 'python3 src/opt2/pipeline2.py' (20 runs):

           2268.51 msec task-clock                #    1.022 CPUs utilized            ( +-  0.25% )
                27      context-switches          #    0.012 K/sec                    ( +-  2.33% )
                 0      cpu-migrations            #    0.000 K/sec                    ( +- 35.04% )
             26132      page-faults               #    0.012 M/sec                    ( +-  0.15% )
   <not supported>      cycles                                                      
   <not supported>      instructions                                                
   <not supported>      branches                                                    
   <not supported>      branch-misses                                               

           2.21934 +- 0.00562 seconds time elapsed  ( +-  0.25% )



## **Compilación en C**

Se trabajaron 2 compilaciones, el método de *Bellman ford* y la *exchange matrix*

### ***Cython***

- Codigo inicial de *Bellman ford*

In [35]:
%%file bf_cython.pyx
import numpy as np 
def bf_negative_cycle_p(graph, node_ini=None, distance_ini=np.inf):
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    if node_ini is None:
        n_nodes = len(graph.nodes())
    else:
        assert node_ini <= len(graph.nodes), f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {len(graph.nodes)}."
        n_nodes = node_ini
            
    n = len(graph.nodes()) + 1
    # Remove nan borders inside graph
    edges = []
    for edge in graph.edges().data():
        if ~np.isnan(edge[2]['weight']):
            edges.append(edge)

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))

    # Initialize distances of nodes and predecessors
    distance= np.ones(n) * distance_ini # Starting distances with infinite values
    distance[n_nodes] = 0  # Starting node has zero distance
    predecessors = np.ones(n) * -1  # Starting predecessors with -1 values
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
        
    return cycle.reverse()

Overwriting bf_cython.pyx


In [36]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("bf_cython.pyx", 
                              compiler_directives={'language_level' : 3})
     )

Overwriting setup.py


In [37]:
%%bash
python3 setup.py build_ext --inplace

Compiling bf_cython.pyx because it changed.
[1/1] Cythonizing bf_cython.pyx
running build_ext
building 'bf_cython' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c bf_cython.c -o build/temp.linux-x86_64-3.8/bf_cython.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/bf_cython.o -o /datos/practica-2-segunda-parte-jesusmb230795/bf_cython.cpython-38-x86_64-linux-gnu.so


**Importando**

In [38]:
import bf_cython
start_time = time.time()
res = bf_cython.bf_negative_cycle_p(G)
end_time = time.time()

In [39]:
secs = end_time-start_time
print("Bellman Ford tomó",secs,"segundos" )

Bellman Ford tomó 0.8361616134643555 segundos


In [40]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate bf_cython.pyx

In [41]:
display(HTML("bf_cython.html"))

- Código perfilado de *Bellman ford*

In [49]:
%%file bf_cython2.pyx
import numpy as np 
def bf_negative_cycle_cc(graph, node_ini=None, unsigned int distance_ini=np.inf):
    
    cdef unsigned int i,n,n_nodes
    
    assert distance_ini>=1, f"La distancia inicial debe de ser mayor o igual a 1. El parámetro fue igual a {distance_ini}"
    
    n_nodes = len(graph.nodes)
    
    if node_ini is not None:
        assert node_ini <= n_nodes, f"El nodo definido es mayor a los del grafo. Deberia de ser menor a {n_nodes}."
        n_nodes = node_ini
                    
    n = n_nodes + 1
    # Remove nan borders inside graph
    edges = [edge for edge in graph.edges().data() if ~np.isnan(edge[2]['weight'])]

    # Add a start node and add zero weighted edges to all other nodes
    for i in range(n-1):
        edges.append((n-1, i, {'weight': 0}))
        
    # Initialize distances of nodes and predecessors
    # https://codingdeekshi.com/initialize-an-array-in-python/
    distance= [distance_ini ]*n
    distance[n_nodes] = 0  
    predecessors = [-1]*n 
    
    for i in range(n):  
        x = -1
        for edge in edges:
            if distance[int(edge[0])] + edge[2]['weight'] < distance[int(edge[1])]:                
                distance[int(edge[1])] = distance[int(edge[0])] + edge[2]['weight']
                predecessors[int(edge[1])] = int(edge[0])
                x = int(edge[1])
        if x == -1:  # If relaxation is not possible, there is no negative cycle
            return None
        
    # Identify negative cycle
    for i in range(n):
        x = predecessors[int(x)]
    cycle = []
    v = x
    while True:
        cycle.append(int(v))
        if v == x and len(cycle) > 1:
            break
        v = predecessors[int(v)]
    
    return cycle.reverse()

Overwriting bf_cython2.pyx


In [50]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(ext_modules = cythonize("bf_cython2.pyx", 
                              compiler_directives={'language_level' : 3})
     )

Overwriting setup.py


In [51]:
%%bash
python3 setup.py build_ext --inplace

Compiling bf_cython2.pyx because it changed.
[1/1] Cythonizing bf_cython2.pyx
running build_ext
building 'bf_cython2' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.8 -c bf_cython2.c -o build/temp.linux-x86_64-3.8/bf_cython2.o
x86_64-linux-gnu-gcc -pthread -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -Wl,-Bsymbolic-functions -Wl,-z,relro -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 build/temp.linux-x86_64-3.8/bf_cython2.o -o /datos/practica-2-segunda-parte-jesusmb230795/bf_cython2.cpython-38-x86_64-linux-gnu.so


**Importando**

In [52]:
import bf_cython2
start_time = time.time()
res = bf_cython2.bf_negative_cycle_cc(G)
end_time = time.time()

In [53]:
secs = end_time-start_time
print("Bellman Ford tomó",secs,"segundos" )

Bellman Ford tomó 0.527334451675415 segundos


In [54]:
%%bash
$HOME/.local/bin/cython --force -3 --annotate bf_cython2.pyx

In [55]:
display(HTML("bf_cython2.html"))

## Bibliografía:

1 [Libro de optimización. 5.2 Herramientas de lenguajes de programación y del sistema operativo para perfilamiento e implementaciones de BLAS](https://itam-ds.github.io/analisis-numerico-computo-cientifico/5.optimizacion_de_codigo/5.2/Herramientas_de_lenguajes_y_del_SO_para_perfilamiento_e_implementaciones_de_BLAS.html)


2[Libro de optimización. 5.3 Compilación a C](https://itam-ds.github.io/analisis-numerico-computo-cientifico/5.optimizacion_de_codigo/5.3/Compilacion_a_C.htmll)